# 1. Preprocessing: translating MoMA files to Pandas

In [1]:
import os, re, glob
import numpy as np
import pandas as pd
from colicycle.MoMAobj import Momaobj
import colicycle.time_mat_operations as tmo
import colicycle.momaprocess as mp

In [23]:
folder_prefix = '../PreProcessed/'

In [24]:
#say where data are located and how many channelss are present in each experiment
data_folder = [[folder_prefix+'20180709_GW296_glucose8aa37_1_MMStack/',2],
            [folder_prefix+'20180711_GW296_glucose37_1_MMStack/',2],
              [folder_prefix+'20180706_GW296_glycerol37_1_MMStack/',2],
               [folder_prefix+'20170327_GW339_temp/',3]
           ]

tosave_folder = folder_prefix


In [25]:
name = [os.path.basename(os.path.normpath(d[0])) for d in data_folder]

In [26]:
for ind, dataset in enumerate(data_folder):
    mom = Momaobj(data_folder=dataset[0],col_nb=dataset[1], no_raw=True)
    
    for i in range(len(mom.pos_list)):
        mom.pos = mom.pos_list[i]
        mom.gl = mom.gl_list[i]

        time_mat_pd = mp.parse_exported(mom.get_momapath())
        time_mat_pd['length'] = time_mat_pd.pixlim.apply(lambda x: x[:,1]-x[:,0])
        time_mat_pd = tmo.essential_props(time_mat_pd)
        #time_mat_pd = mp.moma_cleanup(time_mat_pd)

        time_mat_pd = pd.concat([time_mat_pd, time_mat_pd.apply(lambda row: tmo.exponential_fit(row), axis=1)], axis=1)

        time_mat_pd['pix_max']=time_mat_pd.pixlim.apply(lambda x: x[:,1])
        time_mat_pd['pix_min']=time_mat_pd.pixlim.apply(lambda x: x[:,0])

        time_mat_pd['full_cellcycle'] = time_mat_pd.exit_type.apply(lambda x: True if x=='DIVISION' else False)
        time_mat_pd['full_cellcycle'] = time_mat_pd.exit_type.apply(lambda x: True if x=='DIVISION' else False)
        time_mat_pd[time_mat_pd.born == -1]['full_cellcycle'] = False

        time_mat_pd['mother_id'] = time_mat_pd['mother_id'].fillna(-1).astype(int)

        filename = tosave_folder+'/'+name[ind]+'/step1/step1_time_mat'+'_pos'+mom.pos+'_GL'+mom.gl+'.pkl'
        os.makedirs(os.path.dirname(filename), exist_ok=True)

        time_mat_pd['Ti'] = np.nan
        time_mat_pd['Li'] = np.nan
        time_mat_pd['Li_fit'] = np.nan

        time_mat_pd.to_pickle(filename)

/Users/gw18g940/miniconda3/envs/doubleadder/lib/python3.7/site-packages/scipy/optimize/minpack.py:795: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/gw18g940/miniconda3/envs/doubleadder/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/gw18g940/miniconda3/envs/doubleadder/lib/python3.7/site-packages/scipy/stats/stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/Users/gw18g940/GoogleDrive/PostDocBasel/ColiPublication_GDrive/DoubleAdderCode/colicycle/colicycle/tools_GW.py:10: RuntimeWarning: overflow encountered in power
  ret